In [1]:
# import intake
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
import cmocean.cm as cmo
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
import gribscan
import intake
import numcodecs
import dask
import cartopy.crs as ccrs


cat = intake.open_catalog("/home/b/b381639/nextGems_Cycle2/catalog.yaml")
#run = cat.IFS["tco2559-ng5"]
run = cat.IFS["tco1279-orca025"]
data = run.ICMGG_atm2d.to_dask()

In [3]:
slhf=data.slhf

In [4]:
slhf

<xarray.DataArray 'slhf' (time: 17089, value: 6599680)>
dask.array<open_dataset-139af84cfc9b7a26f3b736798aa1ccb1slhf, shape=(17089, 6599680), dtype=float64, chunksize=(1, 6599680), chunktype=numpy.ndarray>
Coordinates:
    lat      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2022-01-01
Dimensions without coordinates: value
Attributes:
    paramId:                    147
    dataType:                   fc
    numberOfPoints:             6599680
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    gridType:                   reduced_gg
    shortName:                  slhf
    units:                      J m**-2
    name:                       Surface latent heat flux
    cfName:                     surface_upward_latent_heat_flux
    cfVarName:                  slhf
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid

In [5]:
#But fluxes are accumulated from start of the month, or for Jan 2020, it would be Jan 20th. 
#So we need to take the difference from day after to current day. 

yyyys=2020
yyyye=2021
yr=yyyys
#day=4
mms=1
mme=12
cdatearr=[]
dayarr=[]

while yr<=yyyye:
    yyyy=str(yr)
    mth=mms
    if yr==yyyye:
        mthe=mme
    else:
        mthe=12
    while mth<=mthe:
        if mth<10:
            mm='0'+str(mth)
        else:
            mm=str(mth)
        if mth==1 or mth==3 or mth==5 or mth==7 or mth==8 or mth==10 or mth==12:
            daye=31
        elif mth==4 or mth==6 or mth==9 or mth==11:
            daye=30
        if (yr%4)==0 and mth==2:
            daye=29
        elif (yr%4)!=0 and mth==2:
            daye=28
        if yr==2020 and mth==1:
            days=20
        else:
            days=1
        day=days
        tidx=0
        while day<=daye:
            if day<10:
                dd='0'+str(day)
            else:
                dd=str(day)
            cdatearr.append(yyyy+'-'+mm+'-'+dd)
            dayarr.append(day)
            #print(data.str.sel(time=yyyy+'-'+mm+'-'+dd+'T00:00:00.000000000').time)
            day=day+1
            #cidx=cidx+1
        mth=mth+1
    yr=yr+1

print(np.shape(cdatearr)[0]-1)
#print(cdatearr)

711


In [8]:
cidx=0
cdate=cdatearr[cidx]
cdatep=cdatearr[cidx+1]

In [10]:
cdatep

'2020-01-21'

In [13]:
data.sshf.sel(time=cdatep+'T00:00:00.000000000')

<xarray.DataArray 'sshf' (value: 6599680)>
dask.array<getitem, shape=(6599680,), dtype=float64, chunksize=(6599680,), chunktype=numpy.ndarray>
Coordinates:
    lat      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
    time     datetime64[ns] 2020-01-21
Dimensions without coordinates: value
Attributes:
    paramId:                    146
    dataType:                   fc
    numberOfPoints:             6599680
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    gridType:                   reduced_gg
    shortName:                  sshf
    units:                      J m**-2
    name:                       Surface sensible heat flux
    cfName:                     surface_upward_sensible_heat_flux
    cfVarName:                  sshf
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid

In [16]:
data.sshf.sel(time=cdatep+'T00:00:00.000000000').coords['time']

<xarray.DataArray 'time' ()>
array('2020-01-21T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2020-01-21

In [18]:
dayarr[cidx+1]

20

In [19]:
SHFarray=[]
cidx=0
while cidx<=np.shape(cdatearr)[0]-2:
    cdate=cdatearr[cidx]
    cdatep=cdatearr[cidx+1]
    #print(dayarr[cidx+1])
    #dlen=len(dayarr[cidx+1])
    if dayarr[cidx+1]==2 or cidx==0:
        #print('Use only '+cdatep)
        #print('datestamp='+cdate+'T00:00:00.000000000')
#        SHFarray.append(data.sshf.sel(time=cdatep+'T00:00:00.000000000').reset_coords('time', drop=True) /86400)
        SHFarray.append(data.sshf.sel(time=cdatep+'T00:00:00.000000000')/86400)
        #print(SHF.max())
    else:
        #print('Use '+cdatep+' - '+cdate)
        SHFarray.append((data.slhf.sel(time=cdatep+'T00:00:00.000000000')-data.str.sel(time=cdate+'T00:00:00.000000000') )/86400)    
        #SHFarray.append(SHF)
    cidx=cidx+1




In [21]:
SHFarray

[<xarray.DataArray 'sshf' (value: 6599680)>
 dask.array<truediv, shape=(6599680,), dtype=float64, chunksize=(6599680,), chunktype=numpy.ndarray>
 Coordinates:
     lat      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
     lon      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
     time     datetime64[ns] 2020-01-21
 Dimensions without coordinates: value,
 <xarray.DataArray (value: 6599680)>
 dask.array<truediv, shape=(6599680,), dtype=float64, chunksize=(6599680,), chunktype=numpy.ndarray>
 Coordinates:
     lat      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
     lon      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
 Dimensions without coordinates: value,
 <xarray.DataArray (value: 6599680)>
 dask.array<truediv, shape=(6599680,), dtype=float64, chunksize=(6599680,), chunktype=numpy.ndarray>
 Coordinates:
     lat      (value) float64 dask.array<chunksize=(6599680,), meta=np.ndarray>
     lon      (va

In [21]:
xarraySHF=xr.concat(SHFarray,dim='time')

In [20]:
xarraySHF

NameError: name 'xarraySHF' is not defined

In [23]:
import pandas as pd
times = pd.date_range("2020/01/20",periods=711,freq='D',closed='left')
time_da = xr.DataArray(times, [('time', times)])


/tmp/ipykernel_384809/3532847830.py:2: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  times = pd.date_range("2020/01/20",periods=711,freq='D',closed='left')


In [24]:
time_da

<xarray.DataArray (time: 711)>
array(['2020-01-20T00:00:00.000000000', '2020-01-21T00:00:00.000000000',
       '2020-01-22T00:00:00.000000000', ..., '2021-12-28T00:00:00.000000000',
       '2021-12-29T00:00:00.000000000', '2021-12-30T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20 2020-01-21 ... 2021-12-30

In [ ]:
xarraySHF.coords['time']=time_da

In [ ]:
xarraySHF.time

In [7]:
xarraySHF.to_netcdf('/scratch/b/b381639/IFS/SSHF_daily_tco1279-orca025.nc')